### bring File

In [ ]:
import os
import glob


print(os.path.abspath(""))

data_dir = os.path.abspath("../../data/test")
file = glob.glob(f'{data_dir}/*.pdf')
data_dir = os.path.abspath("../../data")
file = glob.glob(f'{data_dir}/*.pdf')


# file = os.path.abspath("../../data/EOS_200D_II_User_Manual.pdf")
file

In [ ]:
# LLAMA_CLOUD_API_KEY, OPENAI_API_KEY, UPSTAGE_API_KEY 로드

from dotenv import load_dotenv
import os

load_dotenv(override=True) # 강제 다시 로드

api_key=os.getenv("OPENAI_API_KEY")

# os.getenv("LLAMA_CLOUD_API_KEY")

up_api_key = os.environ["UPSTAGE_API_KEY"]  # ex: up_xxxYYYzzzAAAbbbCCC

### Llama Parser

In [ ]:
parsing_instruction_ko = '''
당신은 카메라 사용 설명서를 파싱하고 있습니다. 이 문서는 다수의 이미지, 표, 이모티콘을 포함하고 있습니다. 다음 지침을 준수하여 모든 콘텐츠 유형이 명확히 구분되고 잘 파싱되도록 하세요:

### 0. 전체 지침
   - 모든 문서는 마크다운 형식으로, 동일한 문법을 유지하세요.
   - 각 페이지의 번호(바닥글) 은 출력하지 마세요.
   - 모든 텍스트와 이미지는 최대한 원본 형식을 유지하세요.

### 1. 텍스트 파싱
   - 모든 일반 텍스트를 추출하고, 섹션 또는 제목별로 정리하세요(제목이 없는 경우 비워두세요.).
   - 글머리 기호, 번호 목록, 문단과 같은 원본 형식을 유지하세요.
   - 특정 섹션과 함께 제공된 캡션이나 설명도 포함하세요.

### 2. 표 파싱
   - 문서 내의 모든 표를 식별하고 내용을 정확히 추출하세요.
   - 각 표를 행, 열, 헤더를 유지하여 구조화된 형식으로 표현하세요.
   - 표에 제목이나 설명이 있는 경우, 이를 추출하여 표와 함께 포함하세요.

### 3. 이미지 자리 표시
   - 모든 이미지를 "![그림 자리(설명 입력)]"라는 자리 표시 텍스트로 대체하세요.
   - 이미지에 설명이 포함된 경우 이를 추출하여 설명 입력 부분에 텍스트를 추가하세요.
   - 문서 내에서 이미지가 위치했던 위치를 명확히 표시하여 문맥을 유지하세요.

### 4. 이모티콘 처리
   - 영어로 되어있는 이모티콘은 영어 텍스트로 대체하세요.(예시 : "<SET>")
   - 이모티콘이 문서에서 기능적 또는 설명적 목적을 가지고 있다면 추가적인 문맥 정보를 제공하세요. (예시 : "**주의**", "**참고**", "페이지")
   - 위 항목에 포함되지 않는 이모티콘은 "![이모티콘]" 자리 표시로 대체하세요.

### 5. 일반 주의사항
   - 텍스트, 표, 이미지, 이모티콘 간의 경계를 명확히 유지하고, 파싱된 내용이 읽기 쉽고 참조하기 쉽게 하세요.
   - 설명서의 전체 구조와 레이아웃이 원본 문서와 일관성을 유지하도록 하세요.

이 지침을 준수하여 파싱된 콘텐츠가 일관되고 구조적이며 이해하기 쉽게 유지되도록 하세요.

출력 예시)

4. 커버를 닫습니다.
   - 커버가 딸깍 소리를 내며 닫힐 때까지 누르십시오.
   - 전원 스위치를 〈ON〉으로 설정하면 남은 촬영 가능 매수가 LCD 모니터에 표시됩니다.

**주의** 
- 카드 슬롯/배터리실 커버가 열린 상태에서 커버를 강제로 뒤로 더 젖히지 마십시오. 경첩이 파손될 수 있습니다.

**참고** 
- 촬영 가능 매수는 카드의 잔여 용량이나 기록 화질, ISO 감도 등에 따라 달라집니다.  
- [![이모티콘(카메라)]: 카드 없이 셔터를 누름]을 [해제]로 설정하면 촬영 전 카드의 삽입을 잊는 것을 방지할 수 있습니다 (페이지 260).

### 배터리와 카드 분리하기

1. 전원 스위치를 〈OFF〉로 설정합니다 (페이지 41).

2. 커버를 엽니다.
   - 작동 표시등이 꺼진 것을 확인한 후에 커버를 여십시오.
   - [새로운 파일로 저장중...]이 표시되면 커버를 닫으십시오.

3. 배터리를 제거합니다.
   - 화살표 방향으로 배터리 잠금 레버를 눌러 배터리를 제거하십시오.
   - 회로 단락을 방지하기 위해 배터리에 보호 커버 (페이지 35)를 부착하십시오.
'''

In [ ]:
import nest_asyncio
nest_asyncio.apply()

from llama_parse import LlamaParse

# set up parser
parser = LlamaParse(
    use_vendor_multimodal_model=True,
    vendor_multimodal_model_name="openai-gpt4o",
    vendor_multimodal_api_key=api_key,
    num_workers=4,
    result_type="markdown",  # "markdown" and "text" are available
    language="ko",
    parsing_instruction = parsing_instruction_ko,
)

In [ ]:
json_objs_g7x = parser.get_json_result(file[0])
json_list_g7x = json_objs_g7x[0]["pages"]

In [ ]:
# llama parser save
import json

with open('../parsing_result_last/200d_llama_multi.json', 'w', encoding='utf-8') as f:
    json.dump(json_objs_g7x, f, ensure_ascii=False, indent=4)

### upstage parser

In [ ]:
# 100 page split file load
import os
import glob

print(os.path.abspath(""))

data_dir = os.path.abspath("../../../parsing/data/upstage_split")
file = glob.glob(f'{data_dir}/*.pdf')

file

In [ ]:
# 100 chunk data task logic
 
import requests

url = "https://api.upstage.ai/v1/document-ai/document-parse"
headers = {"Authorization": f"Bearer {up_api_key}"}
data_dir = os.path.abspath("../../../parsing/data/upstage_split")
total_num = 800

for i in range(601,total_num,100):
    start_num = i
    fin_num = min(start_num+99, total_num)
    file = f"{data_dir}/page_{start_num}-{fin_num}.pdf"
    files = {"document": open(file, "rb")}
    data = {"output_formats":"['html','markdown']"}
    response = requests.post(url, headers=headers, files=files, data=data)

    with open(f'../parsing_result_last/r6_upstage_{fin_num}.json', 'w', encoding='utf-8') as f:
        json.dump(response.json(), f, ensure_ascii=False, indent=4)
        print(f"{fin_num} file save done")

In [ ]:
def json_data_load(file_path):
    with open(file_path,"r",encoding="utf-8") as file:
        return json.load(file)


def split_data_merge(merge_data, plus_data_path, split_num):
    plus_data = json_data_load(plus_data_path)


    merge_data['content']['html'] += "\n"
    merge_data['content']['html'] += plus_data['content']['html']

    merge_data['content']['markdown'] += "\n"
    merge_data['content']['markdown'] += plus_data['content']['markdown']


    for i in plus_data['elements']:
        i['page'] += (split_num-1)*100

    merge_data['elements'].extend(plus_data['elements'])
    
    return merge_data

In [ ]:
import json
import numpy as np

# JSON 파일 경로 설정
path_dir = os.path.abspath("../parsing_result_last")

cam_name='r50'
file_path = f"{path_dir}/{cam_name}_upstage_100.json"


response = json_data_load(file_path)
response = split_data_merge(response, f"{path_dir}/{cam_name}_upstage_200.json", 2)
response = split_data_merge(response, f"{path_dir}/{cam_name}_upstage_300.json", 3)
response = split_data_merge(response, f"{path_dir}/{cam_name}_upstage_400.json", 4)
response = split_data_merge(response, f"{path_dir}/{cam_name}_upstage_500.json", 5)
response = split_data_merge(response, f"{path_dir}/{cam_name}_upstage_600.json", 6)
response = split_data_merge(response, f"{path_dir}/{cam_name}_upstage_700.json", 7)
response = split_data_merge(response, f"{path_dir}/{cam_name}_upstage_800.json", 8)
# response = split_data_merge(response, f"{path_dir}/{cam_name}_upstage_900.json", 9)
# response = split_data_merge(response, f"{path_dir}/{cam_name}_upstage_1000.json", 10)
# response = split_data_merge(response, f"{path_dir}/{cam_name}_upstage_1059.json", 11)

with open(f'{path_dir}/{cam_name}_upstage_merge.json', 'w', encoding='utf-8') as f:
    json.dump(response, f, ensure_ascii=False, indent=4)

In [ ]:
# extraction image

import os
import glob

data_dir = os.path.abspath("../../data")
file = glob.glob(f'{data_dir}/*.pdf')
file[4]

In [ ]:
import fitz  # PyMuPDF
import os
from tqdm import tqdm

# PDF 파일 경로와 출력 폴더 설정

pdf_path = file[4]  # PDF 파일 경로
cam_name = "m50"
output_folder = os.path.abspath(f"../../parsing/pdf_image/{cam_name}")  # 저장할 폴더 경로

# 출력 폴더가 없으면 생성
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# PDF 열기
pdf_document = fitz.open(pdf_path)

# 해상도 설정 (DPI)
dpi = 300  # 원하는 DPI 값 (300, 600 등)

# 각 페이지를 이미지로 저장
for page_number in tqdm(range(len(pdf_document))):
    page = pdf_document.load_page(page_number)  # 페이지 로드
    pix = page.get_pixmap(dpi=dpi)  # 페이지를 이미지로 렌더링
    output_file = os.path.join(output_folder, f"{cam_name}_page_{page_number + 1}.jpg")
    pix.save(output_file)  # 이미지 저장

pdf_document.close()

In [ ]:
import json


data_dir = os.path.abspath("../../data")
file_path = glob.glob(f'{data_dir}/*.pdf')
file[4]

with open(file_path,"r",encoding="utf-8") as file:
    response = json.load(file)

# figure(image) filtering
filtered_elements = [
    element for element in response.json()['elements']
    if element.get('category') == 'figure'
]

# 새로운 딕셔너리에 저장
filtered_response = {'elements': filtered_elements}


In [ ]:
from PIL import Image

# 이미지 파일 경로
image_path = '/parsing/pdf_image/test2/page_1.png'

# 이미지 열기
for idx,coordinate in enumerate(filtered_response['elements']):
    coordinates = coordinate['coordinates']

    try:
        with Image.open(image_path) as img:
            width, height = img.size

            # 바운딩 박스 좌표 계산
            x_min = int(min(coord['x'] * width for coord in coordinates))
            y_min = int(min(coord['y'] * height for coord in coordinates))
            x_max = int(max(coord['x'] * width for coord in coordinates))
            y_max = int(max(coord['y'] * height for coord in coordinates))

            # 바운딩 박스 크롭
            cropped_image = img.crop((x_min, y_min, x_max, y_max))

            # 결과 이미지 저장
            output_path = f'/parsing/cropped_page_1_{idx}.png'
            cropped_image.save(output_path)
            print(f"Cropped image saved to: {output_path}")

    except FileNotFoundError:
        print(f"File not found: {image_path}")